In [54]:
!pip install openai pinecone pdfplumber tiktoken


In [55]:
!pip install pinecone

In [56]:
import os
import pdfplumber
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
import tiktoken

# ENTER YOUR KEYS HERE:
openai_api_key = "Prefer to use your openAi keys"  # Your OpenAI key
pinecone_api_key = "prefer to use your Pinecone keys"  # Your Pinecone key
index_name = "firstindes"  # Your Pinecone index name
region = "us-east-1"  # Pinecone region

# Initialize clients
client = OpenAI(api_key=openai_api_key)
pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index(index_name)


In [57]:
from google.colab import files

uploaded = files.upload()
file_path = list(uploaded.keys())[0]

print("📄 File uploaded:", file_path)



Saving forsample.pdf to forsample (5).pdf
📄 File uploaded: forsample (5).pdf


In [58]:
def extract_text_from_pdf(path):
    text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

full_text = extract_text_from_pdf(file_path)
print("📚 Total characters extracted:", len(full_text))


📚 Total characters extracted: 175391


In [59]:
def split_text(text, max_tokens=300):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    words = text.split()
    chunks, current_chunk = [], []
    token_count = 0

    for word in words:
        token_count += len(tokenizer.encode(" " + word))
        current_chunk.append(word)
        if token_count >= max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk, token_count = [], 0

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

chunks = split_text(full_text)
print("🧩 Total chunks:", len(chunks))


🧩 Total chunks: 121


In [60]:
from pinecone import Pinecone

# Initialize Pinecone
pc = Pinecone(api_key="prefer to use your own pinecone keys")

# Connect to your manually created index
index_name = "newindex"
index = pc.Index(index_name)
print(" Connected to Pinecone index:", index_name)



 Connected to Pinecone index: newindex


In [61]:

from openai import OpenAI

client = OpenAI(api_key="prefer to use your openAi keys")

for i, chunk in enumerate(chunks):  # your split document text
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=chunk
    )
    embedding = response.data[0].embedding

    # Upload to Pinecone
    index.upsert([
        (f"id-{i}", embedding, {"text": chunk})
    ])

print(" All chunks embedded and uploaded to Pinecone.")


 All chunks embedded and uploaded to Pinecone.


In [62]:
from openai import OpenAI

client = OpenAI(api_key="prefer to us your own keys")

def rag_ask(question, top_k=3):
    # 1. Embed the user question
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=question
    )
    question_embedding = response.data[0].embedding

    # 2. Search Pinecone for similar chunks
    search_response = index.query(
        vector=question_embedding,
        top_k=top_k,
        include_metadata=True
    )

    # 3. Combine the retrieved chunks as context
    context = "\n\n".join([match['metadata']['text'] for match in search_response['matches']])

    # 4. Generate final answer using GPT
    prompt = f"""
You are a helpful assistant answering questions using business documents.

Use the below context to answer the question:
------------------
{context}
------------------

Question: {question}

Answer:"""

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )

    return completion.choices[0].message.content

# Example
print(rag_ask("What is Coca-Cola's policy on sugar and public health?"))
print("Q2:", rag_ask("What risks to reputation does the proposal mention?"))

Coca-Cola has been actively involved in efforts to shift the blame away from sugar as a leading cause of obesity, diabetes, and heart disease. They have attempted to influence public health policy decisions and have funded research groups, such as the Global Energy Balance Network, to promote the idea that lack of exercise is the primary cause of obesity rather than sugar consumption. Coca-Cola has also lobbied against health bills aimed at reducing consumption of their products and has supported organizations that align with their messaging on sugar and public health. The company has been criticized for spreading confusing and misleading messages on a global scale regarding the role of sugar in weight gain and overall public health.
Q2: The proposal mentions the potential reputational risks associated with the public relations efforts to "change the subject" from sugar to exercise, as well as the likely repercussions for the company as these efforts are brought to light. It also discu